In [2]:
import json
import pandas as pd
import os
from tqdm import tqdm

# Base path to StatsBomb open data
base_path = r'C:\Users\nacho\open-data\data'

# Load all competition-season pairs
comps = {
    "La Liga": (11, 27),
    "Premier League": (2, 27),
    "Bundesliga": (9, 27),
    "Serie A": (12, 27),
    "Ligue 1": (7, 27)
}

# Load all match_ids
all_match_ids = []

for comp_name, (comp_id, season_id) in comps.items():
    match_file = os.path.join(base_path, "matches", str(comp_id), f"{season_id}.json")
    with open(match_file, 'r', encoding='utf-8') as f:
        matches = json.load(f)
        for match in matches:
            all_match_ids.append({
                'match_id': match['match_id'],
                'competition': comp_name,
                'home_team': match['home_team']['home_team_name'],
                'away_team': match['away_team']['away_team_name'],
                'match_date': match['match_date']
            })

match_df = pd.DataFrame(all_match_ids)
print(f"✅ Loaded {len(match_df)} matches across 5 leagues.")


✅ Loaded 1823 matches across 5 leagues.


In [3]:
roles_df = pd.read_csv('all_player_roles.csv')
player_role_map = roles_df.set_index('player_id')['role'].to_dict()

In [5]:
from collections import Counter, defaultdict
import pandas as pd
import os
import json
from tqdm import tqdm

minutes_together = Counter()

events_path = os.path.join(base_path, "events")

for _, row in tqdm(match_df.iterrows(), total=len(match_df)):
    match_id = row['match_id']
    event_file = os.path.join(events_path, f"{match_id}.json")

    if not os.path.exists(event_file):
        continue

    with open(event_file, 'r', encoding='utf-8') as f:
        events = json.load(f)

    # For each team: player_id -> (start_minute, end_minute)
    team_player_times = defaultdict(dict)

    # Track which players are starters per team
    starters_per_team = defaultdict(set)

    # Record starters with full 0-90 initially
    for event in events:
        if event['type']['name'] == 'Starting XI':
            team_id = event['team']['id']
            lineup = event['tactics']['lineup']
            for p in lineup:
                pid = p['player']['id']
                team_player_times[team_id][pid] = [0, 90]
                starters_per_team[team_id].add(pid)

    # Process substitutions, but only adjust end time for starters going off
    for event in events:
        if event['type']['name'] == 'Substitution':
            team_id = event['team']['id']
            sub_off = event['player']['id']
            minute = event['minute']

            # Only update if sub_off was a starter
            if sub_off in starters_per_team[team_id]:
                # Update end time to minute subbed off
                team_player_times[team_id][sub_off][1] = minute

            # Do NOT add minutes for sub_on players — ignore them completely

    # Calculate shared minutes only between starters on the same team
    for team_id, player_times in team_player_times.items():
        players = list(player_times.keys())

        for i in range(len(players)):
            for j in range(i + 1, len(players)):
                pid1 = players[i]
                pid2 = players[j]

                start1, end1 = player_times[pid1]
                start2, end2 = player_times[pid2]

                # Calculate overlap
                overlap = max(0, min(end1, end2) - max(start1, start2))

                if overlap > 0:
                    r1 = player_role_map.get(pid1)
                    r2 = player_role_map.get(pid2)

                    if r1 and r2:
                        minutes_together[(r1, r2)] += overlap
                        if r1 != r2:
                            minutes_together[(r2, r1)] += overlap  # symmetric

# Build the minutes matrix
roles = sorted(set(role for pair in minutes_together for role in pair))
minutes_matrix = pd.DataFrame(0.0, index=roles, columns=roles)

for (r1, r2), mins in minutes_together.items():
    minutes_matrix.at[r1, r2] = mins

print(minutes_matrix.round(1))
minutes_matrix.to_csv('minutes matrix.csv')

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [02:39<00:00, 11.44it/s]

                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                   3782.0     23609.0             32882.0   
Anchor Man                          23609.0      7457.0             50735.0   
Attacking Fullback                  32882.0     50735.0             39972.0   
Balanced Fullback                   30224.0     67619.0             63079.0   
Ball Playing CB                     23631.0     48633.0             93625.0   
Ball Winning Midfielder             26576.0     41273.0             56404.0   
Box to Box Midfielder               15105.0     24536.0             66044.0   
Complete Forward                    21997.0     57784.0            116837.0   
Conservative                        14156.0     25023.0             77473.0   
Creative Winger                     15464.0     31855.0             89265.0   
Deep Playmaker                      10257.0     10405.0             62534.0   
Defensive Fullback                  42890.0    10030

In [4]:
# Flatten the matrix and sort by minutes
flat_minutes = minutes_matrix.stack().sort_values(ascending=False)

# Drop duplicate symmetric pairs (e.g., (Defender, Midfielder) and (Midfielder, Defender))
seen = set()
top_pairs = []
for (r1, r2), mins in flat_minutes.items():
    if (r2, r1) not in seen:
        top_pairs.append(((r1, r2), mins))
        seen.add((r1, r2))
    if len(top_pairs) == 10:
        break

# Display the top 10 pairs
print("Top 10 Role Pairs by Minutes Together:")
for i, ((r1, r2), mins) in enumerate(top_pairs, start=1):
    print(f"{i}. {r1} & {r2}: {mins:.1f} minutes")

Top 10 Role Pairs by Minutes Together:
1. Ball Winning Midfielder & Defensive Fullback: 137673.0 minutes
2. Enforcer & Defensive Fullback: 133497.0 minutes
3. Stopper & Defensive Fullback: 119289.0 minutes
4. Defensive Fullback & Complete Forward: 117119.0 minutes
5. Defensive Fullback & Shot Stopper: 116840.0 minutes
6. Complete Forward & Attacking Fullback: 116837.0 minutes
7. Balanced Fullback & Complete Forward: 113137.0 minutes
8. Enforcer & Balanced Fullback: 111683.0 minutes
9. Attacking Fullback & Libero: 111311.0 minutes
10. Balanced Fullback & Stopper: 107032.0 minutes


In [5]:
from collections import Counter
from itertools import combinations
import pandas as pd
import os
import json
from tqdm import tqdm

pair_win_counter = Counter()
pair_total_counter = Counter()

events_path = os.path.join(base_path, "events")

for _, row in tqdm(match_df.iterrows(), total=len(match_df)):
    match_id = row['match_id']
    event_file = os.path.join(events_path, f"{match_id}.json")

    if not os.path.exists(event_file):
        continue

    with open(event_file, 'r', encoding='utf-8') as f:
        events = json.load(f)

    # Extract starting XI lineups and results per team
    starting_xis = {}  # team_id → list of player_ids
    team_results = {}  # team_id → 'win', 'loss', or 'draw'

    # First, find match outcome (home & away teams)
    # Get match info from match_df
    match_row = match_df[match_df['match_id'] == match_id].iloc[0]
    home_team = match_row['home_team']
    away_team = match_row['away_team']

    # Identify teams from events and get team names
    teams_in_match = set()
    for event in events:
        teams_in_match.add(event['team']['id'])
    if len(teams_in_match) != 2:
        # Unexpected, skip this match
        continue
    team_ids = list(teams_in_match)

    # Get goals per team
    goals = {team_id: 0 for team_id in team_ids}
    for event in events:
        if event['type']['name'] == 'Shot' and event.get('shot', {}).get('outcome', {}).get('name') == 'Goal':
            goals[event['team']['id']] += 1

    # Assign results for each team_id
    if goals[team_ids[0]] > goals[team_ids[1]]:
        team_results[team_ids[0]] = 'win'
        team_results[team_ids[1]] = 'loss'
    elif goals[team_ids[0]] < goals[team_ids[1]]:
        team_results[team_ids[0]] = 'loss'
        team_results[team_ids[1]] = 'win'
    else:
        team_results[team_ids[0]] = 'draw'
        team_results[team_ids[1]] = 'draw'

    # Extract Starting XI per team
    for event in events:
        if event['type']['name'] == 'Starting XI':
            team_id = event['team']['id']
            lineup_players = event['tactics']['lineup']
            starting_xis[team_id] = [p['player']['id'] for p in lineup_players]

    # Count win/total for pairs by team
    for team_id, starters in starting_xis.items():
        player_roles = []
        role_counts = Counter()

        for pid in starters:
            role = player_role_map.get(pid)
            if role:
                player_roles.append(role)
                role_counts[role] += 1

        result = team_results.get(team_id)
        if not result:
            continue

        # Different role pairs
        for r1, r2 in combinations(sorted(player_roles), 2):
            pair = tuple(sorted([r1, r2]))
            pair_total_counter[pair] += 1
            if result == 'win':
                pair_win_counter[pair] += 1

        # Same role pairs (diagonal)
        for role, count in role_counts.items():
            if count > 1:
                pairs = count * (count - 1) // 2
                pair_total_counter[(role, role)] += pairs
                if result == 'win':
                    pair_win_counter[(role, role)] += pairs

# Build roles list
roles = sorted(set(role for pair in pair_total_counter for role in pair))

# Create win % matrix
win_matrix = pd.DataFrame(0.0, index=roles, columns=roles)

for (r1, r2), total in pair_total_counter.items():
    wins = pair_win_counter.get((r1, r2), 0)
    win_rate = wins / total if total > 0 else 0
    win_matrix.at[r1, r2] = win_rate
    win_matrix.at[r2, r1] = win_rate  # symmetric
print('minutes')
print(win_matrix.round(3))
# Optionally save
win_matrix.to_csv("win percentage matrix.csv")


100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [01:59<00:00, 15.25it/s]

minutes
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                    0.389       0.374               0.411   
Anchor Man                            0.374       0.389               0.353   
Attacking Fullback                    0.411       0.353               0.540   
Balanced Fullback                     0.333       0.315               0.412   
Ball Playing CB                       0.350       0.350               0.406   
Ball Winning Midfielder               0.338       0.285               0.358   
Box to Box Midfielder                 0.289       0.327               0.376   
Complete Forward                      0.388       0.342               0.475   
Conservative                          0.511       0.427               0.560   
Creative Winger                       0.420       0.357               0.511   
Deep Playmaker                        0.269       0.299               0.503   
Defensive Fullback                    0.323 

In [6]:
# Flatten and sort win rates
flat_win_rates = win_matrix.stack().sort_values(ascending=False)

# Drop symmetric duplicates
seen = set()
top_win_pairs = []
for (r1, r2), win_rate in flat_win_rates.items():
    if (r2, r1) not in seen:
        top_win_pairs.append(((r1, r2), win_rate))
        seen.add((r1, r2))
    if len(top_win_pairs) == 10:
        break

# Display the top 10 pairs
print("Top 10 Role Pairs by Win Percentage:")
for i, ((r1, r2), win_rate) in enumerate(top_win_pairs, start=1):
    print(f"{i}. {r1} & {r2}: {win_rate:.3f}")

Top 10 Role Pairs by Win Percentage:
1. Libero & Holding Midfielder: 0.795
2. Deep Playmaker & Playmaker: 0.691
3. Playmaker & Libero: 0.667
4. Libero & Deep Playmaker: 0.665
5. Libero & Creative Winger: 0.650
6. Libero & Libero: 0.614
7. Holding Midfielder & Conservative: 0.609
8. Libero & Attacking Fullback: 0.601
9. Complete Forward & Libero: 0.589
10. Libero & Conservative: 0.589


In [7]:
from collections import Counter
import pandas as pd
import os
import json
from tqdm import tqdm

# Initialize pass counter
pass_counter = Counter()

events_path = os.path.join(base_path, "events")

# Count passes between roles
for _, row in tqdm(match_df.iterrows(), total=len(match_df)):
    match_id = row['match_id']
    event_file = os.path.join(events_path, f"{match_id}.json")

    if not os.path.exists(event_file):
        continue

    with open(event_file, 'r', encoding='utf-8') as f:
        events = json.load(f)

    starters = set()
    player_to_role = {}

    for event in events:
        if event['type']['name'] == 'Starting XI':
            lineup = event['tactics']['lineup']
            for p in lineup:
                pid = p['player']['id']
                if pid in player_role_map:
                    starters.add(pid)
                    player_to_role[pid] = player_role_map[pid]

    for event in events:
        if event['type']['name'] != 'Pass':
            continue
        passer = event['player']['id']
        receiver = event.get('pass', {}).get('recipient', {}).get('id', None)

        if passer in starters and receiver in starters:
            r1 = player_to_role.get(passer)
            r2 = player_to_role.get(receiver)
            if r1 and r2:
                pair = tuple(sorted((r1, r2)))
                pass_counter[pair] += 1

# Use existing minutes_matrix
roles = minutes_matrix.index.tolist()
pass_matrix_per_90 = pd.DataFrame(0.0, index=roles, columns=roles)

for (r1, r2), count in pass_counter.items():
    if r1 in roles and r2 in roles:
        mins = minutes_matrix.at[r1, r2]
        if mins > 0:
            per_90 = (count / mins) * 90
            pass_matrix_per_90.at[r1, r2] = per_90
            pass_matrix_per_90.at[r2, r1] = per_90  # symmetric

# Final output
print("Passes per 90 (using accurate minutes matrix)")
print(pass_matrix_per_90.round(2))
pass_matrix_per_90.to_csv('pass matrix.csv')

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [01:50<00:00, 16.44it/s]

Passes per 90 (using accurate minutes matrix)
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                    12.95       11.14               12.26   
Anchor Man                            11.14       10.28               10.11   
Attacking Fullback                    12.26       10.11                4.16   
Balanced Fullback                     14.02       10.36                3.57   
Ball Playing CB                        7.20       10.29                9.53   
Ball Winning Midfielder               11.92       11.06               11.58   
Box to Box Midfielder                 11.62       10.93               11.81   
Complete Forward                      12.58        6.91                9.74   
Conservative                           1.81        2.29                3.32   
Creative Winger                       14.68       10.21               14.79   
Deep Playmaker                        13.83       14.54               15.67   
Defens

In [8]:
# Flatten and sort pass rates
flat_pass_rates = pass_matrix_per_90.stack().sort_values(ascending=False)

# Drop symmetric duplicates
seen = set()
top_pass_pairs = []
for (r1, r2), rate in flat_pass_rates.items():
    if (r2, r1) not in seen:
        top_pass_pairs.append(((r1, r2), rate))
        seen.add((r1, r2))
    if len(top_pass_pairs) == 10:
        break

# Display the top 10 pairs
print("Top 10 Role Pairs by Passes per 90 Minutes:")
for i, ((r1, r2), rate) in enumerate(top_pass_pairs, start=1):
    print(f"{i}. {r1} → {r2}: {rate:.2f} passes/90")

Top 10 Role Pairs by Passes per 90 Minutes:
1. Playmaker → Deep Playmaker: 30.14 passes/90
2. Libero → Libero: 24.09 passes/90
3. Deep Playmaker → Libero: 19.87 passes/90
4. Sweeper → Libero: 19.46 passes/90
5. Playmaker → Positional Midfielder: 19.11 passes/90
6. Deep Playmaker → Second Striker: 18.41 passes/90
7. Creative Winger → Playmaker: 17.50 passes/90
8. Second Striker → Creative Winger: 16.90 passes/90
9. Second Striker → Playmaker: 16.79 passes/90
10. Libero → Ball Playing CB: 16.78 passes/90


In [6]:
from collections import Counter
from itertools import combinations
import pandas as pd
import os
import json
from tqdm import tqdm

xg_for_counter = Counter()
pair_total_counter = Counter()

for _, row in tqdm(match_df.iterrows(), total=len(match_df)):
    match_id = row['match_id']

    event_file = os.path.join(events_path, f"{match_id}.json")
    if not os.path.exists(event_file):
        continue

    with open(event_file, 'r', encoding='utf-8') as f:
        events = json.load(f)

    # Extract Starting XI per team from events (correct method)
    starting_xis = {}
    for event in events:
        if event['type']['name'] == 'Starting XI':
            team_id = event['team']['id']
            lineup_players = event['tactics']['lineup']
            player_ids = [p['player']['id'] for p in lineup_players]
            roles = [
                player_role_map.get(pid)
                for pid in player_ids
                if pid in player_role_map
            ]
            if len(roles) >= 2:
                starting_xis[team_id] = roles

    if len(starting_xis) != 2:
        continue  # skip if both teams didn't have valid lineups

    # Get xG for each team
    xg_per_team = {}
    for event in events:
        if event['type']['name'] == 'Shot':
            team_id = event['team']['id']
            xg = event['shot'].get('statsbomb_xg', 0)
            xg_per_team[team_id] = xg_per_team.get(team_id, 0) + xg

    for team_id, roles in starting_xis.items():
        xg_for = xg_per_team.get(team_id, 0)

        role_counts = Counter(roles)

        # Different-role pairs
        for r1, r2 in combinations(sorted(roles), 2):
            pair = tuple(sorted((r1, r2)))
            pair_total_counter[pair] += 1
            xg_for_counter[pair] += xg_for

        # Same-role pairs (e.g. (CM, CM))
        for role, count in role_counts.items():
            if count > 1:
                pairs = count * (count - 1) // 2
                pair_total_counter[(role, role)] += pairs
                xg_for_counter[(role, role)] += xg_for * pairs

# Build role list
unique_roles = sorted(set(role for pair in pair_total_counter for role in pair))

# Create xG For matrix
xg_for_matrix = pd.DataFrame(0.0, index=unique_roles, columns=unique_roles)

for (r1, r2), total in pair_total_counter.items():
    if total == 0:
        continue
    avg_xg = xg_for_counter[(r1, r2)] / total
    xg_for_matrix.at[r1, r2] = avg_xg
    xg_for_matrix.at[r2, r1] = avg_xg  # symmetric

print("xG For matrix:")
print(xg_for_matrix.round(3))

# Optional save
xg_for_matrix.to_csv("xgfor matrix.csv")


100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [02:33<00:00, 11.90it/s]


xG For matrix:
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                    1.413       1.161               1.294   
Anchor Man                            1.161       1.124               1.188   
Attacking Fullback                    1.294       1.188               1.599   
Balanced Fullback                     1.215       1.145               1.333   
Ball Playing CB                       1.254       1.197               1.333   
Ball Winning Midfielder               1.205       1.081               1.201   
Box to Box Midfielder                 1.212       1.104               1.209   
Complete Forward                      1.294       1.185               1.506   
Conservative                          1.488       1.256               1.641   
Creative Winger                       1.471       1.179               1.602   
Deep Playmaker                        1.214       1.255               1.471   
Defensive Fullback                   

In [10]:
# Flatten and sort xG values
flat_xg_values = xg_for_matrix.stack().sort_values(ascending=False)

# Drop symmetric duplicates
seen = set()
top_xg_pairs = []
for (r1, r2), xg in flat_xg_values.items():
    if (r2, r1) not in seen:
        top_xg_pairs.append(((r1, r2), xg))
        seen.add((r1, r2))
    if len(top_xg_pairs) == 10:
        break

# Display the top 10 pairs
print("Top 10 Role Pairs by Average xG For:")
for i, ((r1, r2), xg) in enumerate(top_xg_pairs, start=1):
    print(f"{i}. {r1} & {r2}: {xg:.3f} xG")

Top 10 Role Pairs by Average xG For:
1. Playmaker & Deep Playmaker: 1.972 xG
2. Libero & Playmaker: 1.920 xG
3. Libero & Creative Winger: 1.873 xG
4. Holding Midfielder & Libero: 1.850 xG
5. Libero & Direct Winger: 1.811 xG
6. Libero & Libero: 1.751 xG
7. Attacking Fullback & Libero: 1.737 xG
8. Conservative & Creative Winger: 1.724 xG
9. Libero & Wide Playmaker: 1.722 xG
10. Libero & Deep Playmaker: 1.703 xG


In [7]:
from collections import Counter
from itertools import combinations
import pandas as pd
import os
import json
from tqdm import tqdm

xg_against_counter = Counter()
pair_total_counter = Counter()

for _, row in tqdm(match_df.iterrows(), total=len(match_df)):
    match_id = row['match_id']
    event_file = os.path.join(events_path, f"{match_id}.json")
    if not os.path.exists(event_file):
        continue

    with open(event_file, 'r', encoding='utf-8') as f:
        events = json.load(f)

    # Extract Starting XI from events
    starting_xis = {}
    for event in events:
        if event['type']['name'] == 'Starting XI':
            team_id = event['team']['id']
            lineup_players = event['tactics']['lineup']
            player_ids = [p['player']['id'] for p in lineup_players]
            roles = [
                player_role_map.get(pid)
                for pid in player_ids
                if pid in player_role_map
            ]
            if len(roles) >= 2:
                starting_xis[team_id] = roles

    if len(starting_xis) != 2:
        continue

    # Total xG per team
    xg_per_team = {}
    for event in events:
        if event['type']['name'] == 'Shot':
            team_id = event['team']['id']
            xg = event['shot'].get('statsbomb_xg', 0)
            xg_per_team[team_id] = xg_per_team.get(team_id, 0) + xg

    team_ids = list(starting_xis.keys())
    if len(team_ids) != 2:
        continue

    for team_id in team_ids:
        opponent_id = team_ids[1] if team_id == team_ids[0] else team_ids[0]
        roles = starting_xis[team_id]
        xg_against = xg_per_team.get(opponent_id, 0)

        role_counts = Counter(roles)

        # Different-role pairs
        for r1, r2 in combinations(sorted(roles), 2):
            pair = tuple(sorted((r1, r2)))
            pair_total_counter[pair] += 1
            xg_against_counter[pair] += xg_against

        # Same-role pairs
        for role, count in role_counts.items():
            if count > 1:
                pairs = count * (count - 1) // 2
                pair_total_counter[(role, role)] += pairs
                xg_against_counter[(role, role)] += xg_against * pairs

# Build role list
unique_roles = sorted(set(role for pair in pair_total_counter for role in pair))

# Create xG Against matrix
xg_against_matrix = pd.DataFrame(0.0, index=unique_roles, columns=unique_roles)

for (r1, r2), total in pair_total_counter.items():
    if total == 0:
        continue
    avg_xg_against = xg_against_counter[(r1, r2)] / total
    xg_against_matrix.at[r1, r2] = avg_xg_against
    xg_against_matrix.at[r2, r1] = avg_xg_against  # symmetric

print("xG Against matrix:")
print(xg_against_matrix.round(3))

# Optional save
xg_against_matrix.to_csv("xgagainst matrix.csv")


100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [02:36<00:00, 11.64it/s]

xG Against matrix:
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                    1.136       1.232               1.156   
Anchor Man                            1.232       1.239               1.258   
Attacking Fullback                    1.156       1.258               1.008   
Balanced Fullback                     1.245       1.298               1.153   
Ball Playing CB                       1.251       1.228               1.213   
Ball Winning Midfielder               1.271       1.372               1.298   
Box to Box Midfielder                 1.247       1.286               1.115   
Complete Forward                      1.205       1.261               1.088   
Conservative                          1.001       1.009               0.910   
Creative Winger                       1.085       1.197               1.035   
Deep Playmaker                        1.445       1.320               1.094   
Defensive Fullback               

In [12]:
# Flatten and sort xG Against values (ascending for best defensive pairs)
flat_xg_against = xg_against_matrix.stack().sort_values()

# Drop symmetric duplicates
seen = set()
top_defensive_pairs = []
for (r1, r2), xg in flat_xg_against.items():
    if (r2, r1) not in seen:
        top_defensive_pairs.append(((r1, r2), xg))
        seen.add((r1, r2))
    if len(top_defensive_pairs) == 10:
        break

# Display the top 10 pairs
print("Top 10 Role Pairs by Lowest Average xG Against:")
for i, ((r1, r2), xg) in enumerate(top_defensive_pairs, start=1):
    print(f"{i}. {r1} & {r2}: {xg:.3f} xG Against")

Top 10 Role Pairs by Lowest Average xG Against:
1. Advanced Playmaker & Second Striker: 0.000 xG Against
2. Sweeper Keeper & Sweeper Keeper: 0.000 xG Against
3. Sweeper Keeper & Shot Stopper: 0.000 xG Against
4. Sweeper Keeper & Conservative: 0.000 xG Against
5. Conservative & Conservative: 0.000 xG Against
6. Conservative & Shot Stopper: 0.000 xG Against
7. Shot Stopper & Shot Stopper: 0.000 xG Against
8. Holding Midfielder & Libero: 0.784 xG Against
9. Box to Box Midfielder & Libero: 0.884 xG Against
10. Deep Playmaker & Libero: 0.888 xG Against


In [8]:
# Flatten and sort xG Against values (ascending for best defensive pairs)
flat_xg_against = xg_against_matrix.stack().sort_values()

# Drop symmetric duplicates and exclude pairs with zero minutes
seen = set()
top_defensive_pairs = []

for (r1, r2), xg in flat_xg_against.items():
    if (r2, r1) in seen:
        continue
    minutes = minutes_matrix.at[r1, r2] if r1 in minutes_matrix.index and r2 in minutes_matrix.columns else 0
    if minutes > 0:
        top_defensive_pairs.append(((r1, r2), xg))
        seen.add((r1, r2))
    if len(top_defensive_pairs) == 10:
        break

# Display the top 10 pairs
print("Top 10 Role Pairs by Lowest Average xG Against (min > 0):")
for i, ((r1, r2), xg) in enumerate(top_defensive_pairs, start=1):
    print(f"{i}. {r1} & {r2}: {xg:.3f} xG Against")

Top 10 Role Pairs by Lowest Average xG Against (min > 0):
1. Holding Midfielder & Libero: 0.784 xG Against
2. Box to Box Midfielder & Libero: 0.884 xG Against
3. Deep Playmaker & Libero: 0.888 xG Against
4. Libero & Libero: 0.889 xG Against
5. Deep Playmaker & Playmaker: 0.894 xG Against
6. Libero & Conservative: 0.898 xG Against
7. Conservative & Attacking Fullback: 0.910 xG Against
8. Libero & Attacking Fullback: 0.911 xG Against
9. Holding Midfielder & Conservative: 0.917 xG Against
10. Creative Winger & Libero: 0.920 xG Against


In [13]:
from collections import Counter
import pandas as pd
import os
import json
from tqdm import tqdm

combined_counter = Counter()

events_path = os.path.join(base_path, "events")

for _, row in tqdm(match_df.iterrows(), total=len(match_df)):
    match_id = row['match_id']
    event_file = os.path.join(events_path, f"{match_id}.json")

    if not os.path.exists(event_file):
        continue

    with open(event_file, 'r', encoding='utf-8') as f:
        events = json.load(f)

    starters = set()
    player_to_role = {}

    for event in events:
        if event['type']['name'] == 'Starting XI':
            lineup = event['tactics']['lineup']
            for p in lineup:
                pid = p['player']['id']
                if pid in player_role_map:
                    starters.add(pid)
                    player_to_role[pid] = player_role_map[pid]

    for event in events:
        if event['type']['name'] != 'Pass':
            continue

        passer = event['player']['id']
        receiver = event.get('pass', {}).get('recipient', {}).get('id', None)

        if passer not in starters or receiver not in starters:
            continue

        r1 = player_to_role.get(passer)
        r2 = player_to_role.get(receiver)

        if not r1 or not r2:
            continue

        is_cross = event['pass'].get('cross', False)
        is_shot_assist = event['pass'].get('shot_assist', False)

        if is_cross or is_shot_assist:
            pair = tuple(sorted((r1, r2)))
            combined_counter[pair] += 1  # Count once only

# Initialize matrix
roles = minutes_matrix.index.tolist()
combined_matrix_per_90 = pd.DataFrame(0.0, index=roles, columns=roles)

# Fill matrix with per-90 rates
for (r1, r2), count in combined_counter.items():
    if r1 in roles and r2 in roles:
        mins = minutes_matrix.at[r1, r2]
        if mins > 0:
            per_90 = (count / mins) * 90
            combined_matrix_per_90.at[r1, r2] = per_90
            combined_matrix_per_90.at[r2, r1] = per_90  # symmetric

# Output
print("Crosses or Shot Assists per 90 (by role pair):")
print(combined_matrix_per_90.round(2))
combined_matrix_per_90.to_csv('shotassist matrix.csv')

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [01:49<00:00, 16.62it/s]

Crosses or Shot Assists per 90 (by role pair):
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                     0.79        0.19                0.39   
Anchor Man                             0.19        0.06                0.08   
Attacking Fullback                     0.39        0.08                0.22   
Balanced Fullback                      0.40        0.13                0.24   
Ball Playing CB                        0.23        0.02                0.07   
Ball Winning Midfielder                0.30        0.09                0.16   
Box to Box Midfielder                  0.56        0.14                0.27   
Complete Forward                       1.22        0.25                0.97   
Conservative                           0.00        0.00                0.00   
Creative Winger                        1.35        0.30                0.48   
Deep Playmaker                         0.54        0.18                0.20   
Defen

In [14]:
# Flatten and sort the matrix
flat_combined_rates = combined_matrix_per_90.stack().sort_values(ascending=False)

# Drop symmetric duplicates
seen = set()
top_combined_pairs = []
for (r1, r2), rate in flat_combined_rates.items():
    if (r2, r1) not in seen:
        top_combined_pairs.append(((r1, r2), rate))
        seen.add((r1, r2))
    if len(top_combined_pairs) == 10:
        break

# Display the top 10 pairs
print("Top 10 Role Pairs by Crosses or Shot Assists per 90 Minutes:")
for i, ((r1, r2), rate) in enumerate(top_combined_pairs, start=1):
    print(f"{i}. {r1} → {r2}: {rate:.2f} per 90")

Top 10 Role Pairs by Crosses or Shot Assists per 90 Minutes:
1. Complete Forward → Creative Winger: 1.70 per 90
2. Target Man → Creative Winger: 1.58 per 90
3. Complete Forward → Complete Forward: 1.56 per 90
4. Direct Winger → Complete Forward: 1.48 per 90
5. Second Striker → Complete Forward: 1.48 per 90
6. Creative Winger → Poacher: 1.45 per 90
7. Target Man → Complete Forward: 1.43 per 90
8. Direct Winger → Direct Winger: 1.42 per 90
9. Target Man → Direct Winger: 1.41 per 90
10. Creative Winger → Advanced Playmaker: 1.35 per 90


issues with the solo actions, pair matrices

after do chemistry matrix for attack, defence and both

In [9]:
from collections import defaultdict
import pandas as pd
import os
import json
from tqdm import tqdm

# Define event groups
solo_event_groups = {
    'shots_dribbles': {"Shot", "Dribble"},
    'carries': {"Carry"},
    'defensive_actions': {"Interception", "Clearance", "Block"},
    'ball_recoveries': {"Ball Recovery"},
    'pressures': {"Pressure"},
}

events_path = os.path.join(base_path, "events")
roles = sorted(set(player_role_map.values()))

for group_name, event_types in solo_event_groups.items():
    # Track actions and shared minutes
    action_counts = defaultdict(lambda: defaultdict(int))
    shared_minutes = defaultdict(lambda: defaultdict(float))

    for _, row in tqdm(match_df.iterrows(), total=len(match_df)):
        match_id = row['match_id']
        event_file = os.path.join(events_path, f"{match_id}.json")
        if not os.path.exists(event_file):
            continue

        with open(event_file, 'r', encoding='utf-8') as f:
            events = json.load(f)

        # Parse starters
        player_times = {}
        team_players = defaultdict(list)

        for event in events:
            if event['type']['name'] == 'Starting XI':
                team_id = event['team']['id']
                for p in event['tactics']['lineup']:
                    pid = p['player']['id']
                    if pid in player_role_map:
                        player_times[pid] = {'start': 0, 'end': 90, 'team': team_id, 'role': player_role_map[pid]}
                        team_players[team_id].append(pid)

        # Substitutions
        for event in events:
            if event['type']['name'] == 'Substitution':
                pid = event['player']['id']
                if pid in player_times:
                    player_times[pid]['end'] = event['minute']

        # Minute by minute tracking for accuracy
        for minute in range(90):
            for pid1, info1 in player_times.items():
                if not (info1['start'] <= minute < info1['end']):
                    continue
                role1 = info1['role']
                team1 = info1['team']

                # Shared time with others on same team
                for pid2 in team_players[team1]:
                    if pid1 == pid2:
                        continue
                    info2 = player_times[pid2]
                    if info2['start'] <= minute < info2['end']:
                        role2 = info2['role']
                        shared_minutes[role1][role2] += 1  # 1 minute

        # Count solo actions
        for event in events:
            if event['type']['name'] not in event_types:
                continue
            if 'player' not in event:
                continue
            pid = event['player']['id']
            minute = event['minute']

            if pid not in player_times:
                continue

            role1 = player_times[pid]['role']
            team1 = player_times[pid]['team']

            # Who was on with them at that minute
            for pid2 in team_players[team1]:
                if pid2 == pid or pid2 not in player_times:
                    continue
                info2 = player_times[pid2]
                if info2['start'] <= minute < info2['end']:
                    role2 = info2['role']
                    action_counts[role1][role2] += 1

    # Create final matrix
    df = pd.DataFrame(0.0, index=roles, columns=roles)
    for r1 in roles:
        for r2 in roles:
            mins = shared_minutes[r1][r2]
            if mins > 0:
                val = (action_counts[r1][r2] / mins) * 90
                df.at[r1, r2] = val

    print(f"\n{group_name.upper()} per 90 (non-symmetric):")
    print(df.round(2))
    csv_path = f"{group_name}_per90_matrix.csv"
    df.to_csv(csv_path)

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [02:25<00:00, 12.51it/s]



SHOTS_DRIBBLES per 90 (non-symmetric):
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                     4.78        4.68                4.67   
Anchor Man                             1.35        1.68                1.33   
Attacking Fullback                     1.59        1.73                1.89   
Balanced Fullback                      2.73        2.77                3.22   
Ball Playing CB                        0.82        0.89                0.82   
Ball Winning Midfielder                1.93        2.20                2.14   
Box to Box Midfielder                  2.87        3.62                3.78   
Complete Forward                       5.29        5.20                5.45   
Conservative                           0.04        0.01                0.03   
Creative Winger                        5.80        6.28                7.12   
Deep Playmaker                         2.46        2.78                2.39   
Defensive Fu

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [02:23<00:00, 12.69it/s]



CARRIES per 90 (non-symmetric):
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                    37.58       41.59               40.93   
Anchor Man                            34.79       32.59               33.22   
Attacking Fullback                    35.14       32.59               43.31   
Balanced Fullback                     35.37       35.20               39.80   
Ball Playing CB                       37.60       34.59               36.10   
Ball Winning Midfielder               39.12       37.39               37.42   
Box to Box Midfielder                 38.28       37.06               40.64   
Complete Forward                      26.42       26.65               27.77   
Conservative                          12.24       12.16               12.77   
Creative Winger                       42.47       44.21               48.67   
Deep Playmaker                        40.89       49.93               55.18   
Defensive Fullback 

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [02:21<00:00, 12.89it/s]



DEFENSIVE_ACTIONS per 90 (non-symmetric):
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                     3.44        3.23                3.40   
Anchor Man                             6.00        5.65                5.11   
Attacking Fullback                     5.65        6.21                5.61   
Balanced Fullback                      6.62        6.35                5.66   
Ball Playing CB                        8.64        8.64                8.25   
Ball Winning Midfielder                5.45        5.27                5.41   
Box to Box Midfielder                  5.59        5.06                4.79   
Complete Forward                       2.07        2.15                1.83   
Conservative                           0.06        0.07                0.07   
Creative Winger                        2.81        2.62                2.53   
Deep Playmaker                         4.97        3.87                4.58   
Defensive

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [02:28<00:00, 12.27it/s]



BALL_RECOVERIES per 90 (non-symmetric):
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                     5.72        5.94                5.78   
Anchor Man                             5.26        4.97                5.18   
Attacking Fullback                     4.18        4.30                4.45   
Balanced Fullback                      5.51        4.98                5.37   
Ball Playing CB                        4.11        4.52                4.42   
Ball Winning Midfielder                6.07        6.06                5.93   
Box to Box Midfielder                  6.64        6.30                6.12   
Complete Forward                       3.55        3.76                3.35   
Conservative                           4.12        4.52                4.22   
Creative Winger                        5.80        5.62                5.15   
Deep Playmaker                         6.25        5.69                6.26   
Defensive F

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [02:30<00:00, 12.09it/s]


PRESSURES per 90 (non-symmetric):
                         Advanced Playmaker  Anchor Man  Attacking Fullback  \
Advanced Playmaker                    20.25       21.73               21.49   
Anchor Man                            19.10       17.99               19.40   
Attacking Fullback                    11.42       12.43               12.42   
Balanced Fullback                     15.84       14.61               16.63   
Ball Playing CB                        9.56        9.90               10.03   
Ball Winning Midfielder               23.30       23.42               24.97   
Box to Box Midfielder                 25.07       23.49               23.25   
Complete Forward                      14.82       16.01               14.63   
Conservative                           0.10        0.12                0.10   
Creative Winger                       15.60       16.95               15.91   
Deep Playmaker                        21.50       19.83               19.26   
Defensive Fullbac

In [10]:
    # Extract top 10 role pairs by per-90 value
    flat_df = df.stack().sort_values(ascending=False)

    seen = set()
    top_pairs = []

    for (r1, r2), val in flat_df.items():
        if (r2, r1) in seen:
            continue
        if shared_minutes[r1][r2] > 0:
            top_pairs.append(((r1, r2), val))
            seen.add((r1, r2))
        if len(top_pairs) == 10:
            break

    # Display top 10
    print(f"\nTop 10 Role Pairs by {group_name.replace('_', ' ').title()} per 90 Minutes:")
    for i, ((r1, r2), val) in enumerate(top_pairs, start=1):
        print(f"{i}. {r1} & {r2}: {val:.2f} per 90")


Top 10 Role Pairs by Pressures per 90 Minutes:
1. Ball Winning Midfielder & Playmaker: 25.89 per 90
2. Box to Box Midfielder & False 10: 25.58 per 90
3. Ball Winning Midfielder & Box to Box Midfielder: 25.52 per 90
4. Advanced Playmaker & Playmaker: 25.48 per 90
5. Box to Box Midfielder & Wide Supporter: 25.45 per 90
6. False 10 & Wide Playmaker: 25.19 per 90
7. Box to Box Midfielder & Poacher: 25.11 per 90
8. Box to Box Midfielder & Advanced Playmaker: 25.07 per 90
9. Ball Winning Midfielder & Attacking Fullback: 24.97 per 90
10. Ball Winning Midfielder & Defensive Winger: 24.51 per 90


In [12]:
# Load and print top 10 for each matrix
for group_name in solo_event_groups.keys():
    file_path = f"{group_name}_per90_matrix.csv"
    df = pd.read_csv(file_path, index_col=0)

    flat_df = df.stack().sort_values(ascending=False)
    seen = set()
    top_pairs = []

    for (r1, r2), val in flat_df.items():
        if (r2, r1) in seen:
            continue
        if val > 0:
            top_pairs.append(((r1, r2), val))
            seen.add((r1, r2))
        if len(top_pairs) == 10:
            break

    print(f"\nTop 10 Role Pairs by {group_name.replace('_', ' ').title()} per 90 Minutes:")
    for i, ((r1, r2), val) in enumerate(top_pairs, start=1):
        print(f"{i}. {r1} & {r2}: {val:.2f} per 90")


Top 10 Role Pairs by Shots Dribbles per 90 Minutes:
1. Creative Winger & Libero: 8.05 per 90
2. Creative Winger & Playmaker: 7.68 per 90
3. Creative Winger & Conservative: 7.15 per 90
4. Creative Winger & Attacking Fullback: 7.12 per 90
5. Creative Winger & Complete Forward: 7.09 per 90
6. Creative Winger & Second Striker: 7.02 per 90
7. Creative Winger & Deep Playmaker: 6.81 per 90
8. Creative Winger & Creative Winger: 6.80 per 90
9. Creative Winger & Positional Midfielder: 6.68 per 90
10. Creative Winger & Sweeper: 6.67 per 90

Top 10 Role Pairs by Carries per 90 Minutes:
1. Deep Playmaker & Playmaker: 73.06 per 90
2. Deep Playmaker & Libero: 63.77 per 90
3. Deep Playmaker & Conservative: 63.38 per 90
4. Playmaker & Libero: 62.69 per 90
5. Playmaker & Conservative: 58.40 per 90
6. Libero & Wide Playmaker: 57.65 per 90
7. Playmaker & Positional Midfielder: 55.37 per 90
8. Playmaker & Playmaker: 55.34 per 90
9. Playmaker & Creative Winger: 55.22 per 90
10. Playmaker & Second Striker: 

In [18]:
# Choose the role you want to inspect
target_role = "Ball Winning Midfielder"  # ← Change this to any role you want

# Load and print all values for the target role across each matrix
for group_name in solo_event_groups.keys():
    file_path = f"{group_name}_per90_matrix.csv"
    df = pd.read_csv(file_path, index_col=0)

    if target_role not in df.index:
        print(f"\n{target_role} not found in {group_name} matrix.")
        continue

    row_values = df.loc[target_role]
    sorted_row = row_values[row_values > 0].sort_values(ascending=False)

    print(f"\n{group_name.replace('_', ' ').title()} per 90 Minutes for '{target_role}':")
    for role, val in sorted_row.items():
        print(f"- {target_role} & {role}: {val:.2f} per 90")


Shots Dribbles per 90 Minutes for 'Ball Winning Midfielder':
- Ball Winning Midfielder & Libero: 2.86 per 90
- Ball Winning Midfielder & Positional Midfielder: 2.62 per 90
- Ball Winning Midfielder & Creative Winger: 2.47 per 90
- Ball Winning Midfielder & Conservative: 2.44 per 90
- Ball Winning Midfielder & Ball Playing CB: 2.27 per 90
- Ball Winning Midfielder & Complete Forward: 2.22 per 90
- Ball Winning Midfielder & Balanced Fullback: 2.22 per 90
- Ball Winning Midfielder & Deep Playmaker: 2.22 per 90
- Ball Winning Midfielder & Anchor Man: 2.20 per 90
- Ball Winning Midfielder & Poacher: 2.20 per 90
- Ball Winning Midfielder & Wide Playmaker: 2.18 per 90
- Ball Winning Midfielder & Enforcer: 2.14 per 90
- Ball Winning Midfielder & Attacking Fullback: 2.14 per 90
- Ball Winning Midfielder & Sweeper Keeper: 2.09 per 90
- Ball Winning Midfielder & Stopper: 2.09 per 90
- Ball Winning Midfielder & Target Man: 2.08 per 90
- Ball Winning Midfielder & Second Striker: 2.03 per 90
- Ball

In [14]:
# Load and print top 10 based on combined two-way values
for group_name in solo_event_groups.keys():
    file_path = f"{group_name}_per90_matrix.csv"
    df = pd.read_csv(file_path, index_col=0)

    combined_values = {}
    roles = df.index.tolist()

    for r1 in roles:
        for r2 in roles:
            val1 = df.at[r1, r2]
            val2 = df.at[r2, r1]
            if r1 == r2:
                combined = val1 * 2  # Diagonal: double the self-value
            else:
                combined = val1 + val2
            if combined > 0:
                pair = tuple(sorted((r1, r2)))
                combined_values[pair] = combined

    # Sort by combined value descending
    sorted_pairs = sorted(combined_values.items(), key=lambda x: x[1], reverse=True)

    # Print top 10
    print(f"\nTop 10 Role Pairs by Combined {group_name.replace('_', ' ').title()} per 90 Minutes:")
    for i, ((r1, r2), val) in enumerate(sorted_pairs[:10], start=1):
        print(f"{i}. {r1} & {r2}: {val:.2f} per 90")


Top 10 Role Pairs by Combined Shots Dribbles per 90 Minutes:
1. Creative Winger & Creative Winger: 13.61 per 90
2. Complete Forward & Creative Winger: 12.97 per 90
3. Creative Winger & Wide Playmaker: 12.92 per 90
4. Creative Winger & Direct Winger: 11.59 per 90
5. Creative Winger & Second Striker: 11.55 per 90
6. Advanced Playmaker & Creative Winger: 11.02 per 90
7. Direct Winger & Direct Winger: 10.89 per 90
8. Creative Winger & Playmaker: 10.69 per 90
9. Complete Forward & Direct Winger: 10.62 per 90
10. Advanced Playmaker & Direct Winger: 10.50 per 90

Top 10 Role Pairs by Combined Carries per 90 Minutes:
1. Deep Playmaker & Playmaker: 137.84 per 90
2. Libero & Playmaker: 117.85 per 90
3. Deep Playmaker & Libero: 115.52 per 90
4. Playmaker & Playmaker: 110.68 per 90
5. Playmaker & Positional Midfielder: 107.86 per 90
6. Creative Winger & Libero: 107.67 per 90
7. Libero & Libero: 107.06 per 90
8. Creative Winger & Playmaker: 106.32 per 90
9. Playmaker & Second Striker: 104.17 per 9